Installing Packages

In [ ]:
#ref : https://pypi.org/project/onnx2tf/
!sudo apt-get -y update
!sudo apt-get -y install python3-pip
!sudo apt-get -y install python-is-python3
!wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz \
  && tar -zxvf flatc.tar.gz \
  && sudo chmod +x flatc \
  && sudo mv flatc /usr/bin/
!pip install -U pip \
  && pip install tensorflow==2.15.0 \
  && pip install -U onnx==1.15.0 \
  && python -m pip install onnx_graphsurgeon \
        --index-url https://pypi.ngc.nvidia.com \
  && pip install -U onnxruntime==1.16.3 \
  && pip install -U onnxsim==0.4.33 \
  && pip install -U simple_onnx_processing_tools \
  && pip install -U onnx2tf \
  && pip install -U protobuf==3.20.3 \
  && pip install -U h5py==3.7.0 \
  && pip install -U psutil==5.9.5 \
  && pip install -U ml_dtypes==0.2.0 \
  && pip install -U install tflite-runtime

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,344 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [50.4 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,321 kB]
Hit:13 https://ppa.launchpadcontent.net

importing modules

In [ ]:
import math
import onnxruntime
from google.colab import drive
drive.mount("/content/drive")
from PIL import Image
import requests
import matplotlib.pyplot as plt
import tensorflow
import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
import tflite_runtime.interpreter as tflite
torch.set_grad_enabled(False);
model_pytorch = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model_pytorch.eval();

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using cache found in /root/.cache/torch/hub/facebookresearch_detr_main
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
test_input=torch.rand(1,3,224,224)

In [ ]:
np.array(test_input).shape

(1, 3, 224, 224)

In [ ]:
import numpy as np
torch_test=model_pytorch(test_input)
model_path = '/content/drive/MyDrive/detr.onnx'
session = onnxruntime.InferenceSession(model_path)
output_onnx = session.run(None, {'samples': np.array(test_input)})


In [ ]:
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path="/content/drive/MyDrive/saved_model/detr_float32.tflite")
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
input_shape = input_details[0]['shape']

interpreter.set_tensor(input_details[0]['index'], np.array(test_input).reshape(*input_shape))
interpreter.invoke()
output_details = interpreter.get_output_details()
output_tflite= interpreter.get_tensor(output_details[0]['index'])
bounding_boxes = interpreter.get_tensor(output_details[1]['index'])

In [ ]:
output_torch=model_pytorch(test_input)

In [ ]:
print (output_tflite,bounding_boxes)

[[[-14.985021    -0.52629656  -8.675054   ...  -8.225269    -1.1077448
     6.6768036 ]
  [-14.215692    -1.6252728  -10.598352   ...  -8.010204    -0.7074964
     6.633631  ]
  [-15.109755    -1.1649247   -8.084171   ...  -8.807238    -0.6220833
     6.5877557 ]
  ...
  [-14.5326      -2.13282    -10.370603   ...  -7.9757166   -1.1393679
     6.9669147 ]
  [-16.045654    -0.606561   -11.262517   ... -10.472921    -3.497094
     7.521666  ]
  [-14.178555    -1.6533346  -10.508654   ...  -7.559161    -0.9246682
     6.6196146 ]]] [[[0.01943071 0.30644268 0.03944086 0.46804935]
  [0.41612643 0.03426604 0.17206517 0.06863253]
  [0.05781765 0.90198326 0.10894658 0.11970572]
  [0.48469886 0.02398303 0.13485934 0.04862932]
  [0.2599236  0.03588592 0.4525646  0.07389977]
  [0.24467325 0.30310404 0.4224706  0.68301344]
  [0.36471903 0.03708509 0.09555915 0.07104117]
  [0.0972491  0.86648536 0.09924681 0.09710261]
  [0.09789863 0.04205734 0.12125843 0.07023886]
  [0.049846   0.05729618 0.098574

In [ ]:
print (output_onnx)

[array([[[-15.152715 ,  -0.5435135,  -8.769239 , ...,  -8.221522 ,
          -1.055368 ,   6.7385793],
        [-14.338001 ,  -1.5487102, -10.583308 , ...,  -8.175468 ,
          -0.9829855,   6.738666 ],
        [-15.170836 ,  -1.1180991,  -8.236302 , ...,  -8.959717 ,
          -0.579426 ,   6.7321973],
        ...,
        [-15.975447 ,  -2.2527103, -10.472634 , ...,  -8.517937 ,
          -2.6731963,   7.8229156],
        [-16.150944 ,  -0.5853299, -11.386029 , ..., -10.318916 ,
          -3.5313306,   7.646755 ],
        [-14.528879 ,  -1.697358 , -10.808172 , ...,  -8.10745  ,
          -1.3556025,   6.955521 ]]], dtype=float32), array([[[0.01829505, 0.3170718 , 0.03726491, 0.50810605],
        [0.42694354, 0.03459549, 0.18571362, 0.06944567],
        [0.06357586, 0.9401029 , 0.11135519, 0.08983523],
        [0.53862435, 0.02441123, 0.18828896, 0.04993629],
        [0.28093433, 0.03406265, 0.52951115, 0.07015678],
        [0.2658258 , 0.44713163, 0.54426837, 0.9027463 ],
        

In [ ]:
print (output_torch["pred_logits"],output_torch["pred_boxes"])

tensor([[[-15.1527,  -0.5435,  -8.7693,  ...,  -8.2215,  -1.0554,   6.7386],
         [-14.3380,  -1.5487, -10.5833,  ...,  -8.1755,  -0.9830,   6.7387],
         [-15.1708,  -1.1181,  -8.2363,  ...,  -8.9597,  -0.5794,   6.7322],
         ...,
         [-15.9754,  -2.2527, -10.4726,  ...,  -8.5179,  -2.6732,   7.8229],
         [-16.1509,  -0.5853, -11.3860,  ..., -10.3189,  -3.5313,   7.6468],
         [-14.5289,  -1.6974, -10.8082,  ...,  -8.1074,  -1.3556,   6.9555]]]) tensor([[[0.0183, 0.3171, 0.0373, 0.5081],
         [0.4269, 0.0346, 0.1857, 0.0694],
         [0.0636, 0.9401, 0.1114, 0.0898],
         [0.5386, 0.0244, 0.1883, 0.0499],
         [0.2809, 0.0341, 0.5295, 0.0702],
         [0.2658, 0.4471, 0.5443, 0.9027],
         [0.4007, 0.0306, 0.1450, 0.0612],
         [0.0899, 0.8977, 0.0899, 0.0965],
         [0.0907, 0.0422, 0.1141, 0.0713],
         [0.0463, 0.0537, 0.0918, 0.0920],
         [0.0908, 0.9781, 0.1389, 0.0441],
         [0.0186, 0.7891, 0.0378, 0.2409],
      